<a href="https://colab.research.google.com/github/ssv273/Recom_systems/blob/master/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install implicit

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from tqdm.notebook import tqdm

In [4]:
data = pd.read_csv('/content/drive/My Drive/Romendation_systems/Lesson_2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [0]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [0]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    proba = items_weights['weight'].to_list()

    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=proba)
    
    
    return recs.tolist()

In [0]:
tmp_df = data_train.groupby('item_id')['sales_value'].sum().reset_index()
tmp_df = tmp_df[tmp_df['sales_value'] >= 1]
tmp_df['log_sales_value'] = np.log(tmp_df['sales_value'])
tmp_df['weight'] = tmp_df['log_sales_value'] / tmp_df['log_sales_value'].sum()
tmp_df = tmp_df[['item_id', 'weight']]

In [9]:
tmp_df['weight'].sum()

0.9999999999999999

In [10]:
%%time

# your_code
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(tmp_df, n=5))

CPU times: user 13.2 s, sys: 160 ms, total: 13.4 s
Wall time: 13.4 s


In [11]:
result.head()

,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9837484, 5567009, 6391524, 82526, 12385476]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12171810, 10198378, 943612, 834929, 850450]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1032459, 1134454, 10181318, 1108614, 2045831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[12582377, 1044870, 883358, 869621, 853987]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[883030, 9396772, 9297272, 876952, 1043084]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

#### 2.1 Item-Item Recommender / ItemKNN

In [0]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,819255,819304,819308,819330,819518,819594,819643,819765,819840,819845,819927,819978,820082,820122,820165,820291,820301,820321,820361,820486,820518,820560,820701,820895,821025,821083,821200,821209,821219,821344,...,13512965,13671759,13672065,13777104,13841744,13842088,13842090,13842214,13842224,13877192,13945141,13945244,13987135,14025185,14043817,14043823,14043825,14043826,14050436,14050460,14050461,14077333,14106445,15452677,15452812,15506577,15511891,15596279,15596488,15596515,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 2.62 s, sys: 37.2 ms, total: 2.66 s
Wall time: 1.98 s


In [16]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 81.2 ms, sys: 1.02 ms, total: 82.2 ms
Wall time: 81.7 ms


In [17]:
result.head(2)

,user_id,actual,weighted_random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9837484, 5567009, 6391524, 82526, 12385476]","[981760, 1127831, 1098066, 826249, 878996]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12171810, 10198378, 943612, 834929, 850450]","[981760, 995242, 1029743, 840361, 961554]"


#### 4.2 Косинусное сходство и CosineRecommender

In [18]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 2.65 s, sys: 32.1 ms, total: 2.68 s
Wall time: 1.96 s


In [19]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 94.5 ms, sys: 988 µs, total: 95.5 ms
Wall time: 100 ms


In [20]:
result.head(2)

,user_id,actual,weighted_random_recommendation,itemitem,cosine
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9837484, 5567009, 6391524, 82526, 12385476]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12171810, 10198378, 943612, 834929, 850450]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]"


#### 4.3 TF-IDF взвешивание и TFIDFRecommender

In [21]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 2.65 s, sys: 24.1 ms, total: 2.67 s
Wall time: 1.99 s


In [22]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 93.8 ms, sys: 0 ns, total: 93.8 ms
Wall time: 94.2 ms


#### 4.4 Трюк

In [23]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=False)


CPU times: user 2.6 s, sys: 34.7 ms, total: 2.63 s
Wall time: 1.92 s


In [24]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 78 ms, sys: 0 ns, total: 78 ms
Wall time: 78.4 ms


In [25]:
result.head(2)

,user_id,actual,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9837484, 5567009, 6391524, 82526, 12385476]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12171810, 10198378, 943612, 834929, 850450]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [0]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])

    
    bought_list = bought_list  # Тут нет [:k] !!
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [0]:
wrr_precision_at_k = 0
itemitem_precision_at_k = 0
cosine_precision_at_k = 0
tfidf_precision_at_k = 0
own_purchases_precision_at_k = 0

In [28]:
k = 5
for i in tqdm(result.index):
  wrr_precision_at_k += precision_at_k(result.iloc[i][2], result.iloc[i][1], k=k)
  itemitem_precision_at_k += precision_at_k(result.iloc[i][3], result.iloc[i][1], k=k)
  cosine_precision_at_k += precision_at_k(result.iloc[i][4], result.iloc[i][1], k=k)
  tfidf_precision_at_k += precision_at_k(result.iloc[i][5], result.iloc[i][1], k=k)
  own_purchases_precision_at_k += precision_at_k(result.iloc[i][6], result.iloc[i][1], k=k)



In [29]:
print(f'precision weighted_random_recommendation@{k} = {wrr_precision_at_k / result.shape[0]:.4%}')
print(f'precision itemitem@{k} = {itemitem_precision_at_k / result.shape[0]:.4%}')
print(f'precision cosine@{k} = {cosine_precision_at_k / result.shape[0]:.4%}')
print(f'precision tfidf@{k} = {tfidf_precision_at_k / result.shape[0]:.4%}')
print(f'precision own_purchases@{k} = {own_purchases_precision_at_k / result.shape[0]:.4%}')

precision weighted_random_recommendation@5 = 0.2351%
precision itemitem@5 = 3.3595%
precision cosine@5 = 3.5260%
precision tfidf@5 = 3.6141%
precision own_purchases@5 = 17.9693%


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

#### Baselines

In [0]:
N = 5000

In [0]:
res_top_n = data_train.groupby('item_id').agg({'sales_value':'sum'}).reset_index().\
                               sort_values('sales_value',ascending=False).head(N)['item_id'].to_list()

In [0]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [33]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,weighted_random_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9837484, 5567009, 6391524, 82526, 12385476]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[8090541, 13115548, 10351524, 970365, 943088]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[12171810, 10198378, 943612, 834929, 850450]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[7025275, 937928, 1139184, 850679, 1092945]"


In [34]:
%%time

items = res_top_n

result['random_recommendation_top_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 1.01 s, sys: 9.25 ms, total: 1.02 s
Wall time: 1.02 s


In [35]:
result.iloc[0]

user_id                                                                           1
actual                            [821867, 834484, 856942, 865456, 889248, 90795...
weighted_random_recommendation         [9837484, 5567009, 6391524, 82526, 12385476]
itemitem                                 [981760, 1127831, 1098066, 826249, 878996]
cosine                                   [981760, 1127831, 1098066, 878996, 826249]
tfidf                                    [981760, 1127831, 1098066, 826249, 878996]
own_purchases                           [999999, 1082185, 1029743, 995785, 1004906]
random_recommendation                 [8090541, 13115548, 10351524, 970365, 943088]
random_recommendation_top_5000           [5580525, 996009, 965693, 1037337, 822524]
Name: 0, dtype: object

In [0]:
random_recommendation_at_k = 0
random_recommendation_top_5000_at_k = 0

In [37]:
k = 5
for i in tqdm(result.index):
  random_recommendation_at_k += precision_at_k(result.iloc[i][7], result.iloc[i][1], k=k)
  random_recommendation_top_5000_at_k += precision_at_k(result.iloc[i][8], result.iloc[i][1], k=k)

In [38]:
print(f'precision random_recommendation@{k} = {random_recommendation_at_k / result.shape[0]:.4%}')
print(f'precision random_recommendation_top_5000@{k} = {random_recommendation_top_5000_at_k / result.shape[0]:.4%}')

precision random_recommendation@5 = 0.5485%
precision random_recommendation_top_5000@5 = 0.5093%


Как мы видим - случайный выбор из 500 товаров дает результат хуже, хотя по идее должен давать лучше

In [0]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [40]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 97.8 ms, sys: 896 µs, total: 98.7 ms
Wall time: 121 ms


Эта рекомендация никак не зависит от того отобрали ли мы предварительно топ каких-то товаров или нет, за исключением времени выполнения.

In [0]:
def weighted_random_recommendation(items_weights, n=5):

    proba = items_weights['weight'].to_list()

    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=proba)
    
    
    return recs.tolist()

In [0]:
tmp_df = data_train.groupby('item_id')['sales_value'].sum().reset_index().sort_values('sales_value',ascending=False).head(N)
tmp_df = tmp_df[tmp_df['sales_value'] >= 1]
tmp_df['log_sales_value'] = np.log(tmp_df['sales_value'])
tmp_df['weight'] = tmp_df['log_sales_value'] / tmp_df['log_sales_value'].sum()
tmp_df = tmp_df[['item_id', 'weight']]

In [43]:
tmp_df['weight'].sum()

1.0

In [44]:
%%time

result['weighted_random_recommendation_top_5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(tmp_df, n=5))

CPU times: user 1.18 s, sys: 4.87 ms, total: 1.18 s
Wall time: 1.2 s


In [0]:
wrr_precision_top_5000_at_k = 0

In [46]:
k = 5
for i in tqdm(result.index):
  wrr_precision_top_5000_at_k += precision_at_k(result.iloc[i][9], result.iloc[i][1], k=k)

In [47]:
print(f'precision weighted_random_recommendation_top_5000@{k} = {wrr_precision_top_5000_at_k / result.shape[0]:.4%}')
print(f'precision weighted_random_recommendation@{k} = {wrr_precision_at_k / result.shape[0]:.4%}')

precision weighted_random_recommendation_top_5000@5 = 8.5602%
precision weighted_random_recommendation@5 = 0.2351%


Метрика значительно улучшилась

## Choose K

In [0]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [0]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [50]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,819255,819304,819308,819330,819518,819594,819643,819765,819840,819845,819927,819978,820082,820122,820165,820291,820301,820321,820361,820486,820518,820560,820701,820895,821025,821083,821200,821209,821219,821344,...,13512965,13671759,13672065,13777104,13841744,13842088,13842090,13842214,13842224,13877192,13945141,13945244,13987135,14025185,14043817,14043823,14043825,14043826,14050436,14050460,14050461,14077333,14106445,15452677,15452812,15506577,15511891,15596279,15596488,15596515,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

 ItemKNN

# К сожалению код ниже крашит почему-то ноутбук, не могу посчитать. Один раз получилось, но повторить нет. Колаб почему-то падает с неизвестной ошибкой

In [0]:
for knn in range(20):

  model = ItemItemRecommender(K=knn, num_threads=4) # K - кол-во билжайших соседей

  model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  result[f'itemitem_at_{knn}'] = result['user_id'].\
      apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                      model.recommend(userid=userid_to_id[x], 
                                      user_items=sparse_user_item,   # на вход user-item matrix
                                      N=5, 
                                      filter_already_liked_items=True, 
                                      filter_items=None, 
                                      recalculate_user=True)])

In [0]:
result.head()

In [0]:
k = 5
prc_ItemKNN = []
for i in tqdm(result.index):
  for j in result.columns[2:]:
      prc_ItemKNN.append()
  wrr_precision_top_5000_at_k += precision_at_k(result.iloc[i][9], result.iloc[i][1], k=k)

In [0]:
result.columns[2:]

Index(['itemitem_at_0', 'itemitem_at_1', 'itemitem_at_2', 'itemitem_at_3',
       'itemitem_at_4', 'itemitem_at_5', 'itemitem_at_6', 'itemitem_at_7',
       'itemitem_at_8', 'itemitem_at_9', 'itemitem_at_10', 'itemitem_at_11',
       'itemitem_at_12', 'itemitem_at_13', 'itemitem_at_14', 'itemitem_at_15',
       'itemitem_at_16', 'itemitem_at_17', 'itemitem_at_18', 'itemitem_at_19'],
      dtype='object')

In [0]:
result.iloc[0]

user_id                                                           1
actual            [821867, 834484, 856942, 865456, 889248, 90795...
itemitem_at_0            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_1            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_2            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_3            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_4            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_5            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_6            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_7            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_8            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_9            [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_10           [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_11           [981760, 1127831, 1098066, 826249, 878996]
itemitem_at_12           [981760, 1127831, 10980

In [0]:
# your_code